# Normal in vitro clock

The circadian clock of cyanobacteria *Synechococcus elongatus PCC 7942* can be reconstructed *in vitro*.
This protocol samples such an *in vitro* clock with uniform time intervals.

## Initialize the protocol

In [1]:
# current working dir is /var/lib/jupyter/notebooks
import opentrons.execute
from opentrons import protocol_api
import numpy as np
from src.pipette_viscous import transfer_viscous,aspirate_viscous, dispense_viscous, calibrated_viscous
import src.scheduler as scheduler
from src.evap_standard_curve import *
import time

# start the protocol context
protocol = opentrons.execute.get_protocol_api("2.13")

# home is required
protocol.set_rail_lights(False)
protocol.home()

## Load labware

- The incubator is on slot 11
- The alluminum rack is mounted to the incubator
- The GEB tip racks are on slot 7, 8, and 9
- The Bio-rad PCR plate is on slot 5
- The P10 1st gen pipette is on the left

In [2]:
incubator = protocol.load_module("temperature module", 11)

rack = incubator.load_labware("eppendorf_24_aluminumblock_1500ul")    # the rack is mounted upon the temp. module (don't specify slot)

tip_rack_1 = protocol.load_labware("geb_taller_96_tiprack_10ul", '7')
tip_rack_2 = protocol.load_labware("geb_taller_96_tiprack_10ul", '8')
tip_rack_3 = protocol.load_labware("geb_taller_96_tiprack_10ul", '9')

plate = protocol.load_labware("biorad_96_wellplate_200ul_pcr", '5')

pipette = protocol.load_instrument("p10_single", "left", 
                                   tip_racks = [tip_rack_1, tip_rack_2, tip_rack_3])

## Sampling and liquid settings


In [3]:
SAMPLE_INT = 4 * 60    # time interval in minutes
SAMPLE_STEPS = 18    # plus the initial sampling, in total sample SAMPLE_STEPS + 1 times
SAMPLE_VOL = 9    # ul, volume per sample
SAMPLE_OFFSET = 0    # define the starting well
HYDRATE_INT = 4 * 60    # time interval to re-hydrate the samples to avoid drying down
HYDRATE_VOL = evap_vol_4h(SAMPLE_VOL * 4 / 3)    # 5.1

RXN_TUBE = rack.wells_by_name()["A1"]
DYE_TUBE = rack.wells_by_name()["A3"]
WAT_TUBE = rack.wells_by_name()["A5"]

print(f"Volume of water adding to sample is {HYDRATE_VOL} ul")

Volume of water adding to sample is 6.1 ul


The below cell calculates minimal volume in each tube (overlay ~60 ul mineral oil on the reaction solution and ~40 ul on the dye)

In [4]:
rxn_vol = SAMPLE_VOL * (SAMPLE_STEPS + 1) + 30
dye_vol = SAMPLE_VOL / 3 * (SAMPLE_STEPS + 1) + 30
per_sampling = int(SAMPLE_INT / HYDRATE_INT)
wat_vol = sum(range(1, SAMPLE_STEPS + 1)) * per_sampling * HYDRATE_VOL + 30

dye_vol = int(np.floor(dye_vol))
wat_vol = int(np.floor(wat_vol))

print(f"Reaction tube at A1, {rxn_vol} ul")
print(f"Dye tube at A3, {dye_vol} ul")
print(f"Water tube at A5, {wat_vol} ul")

Reaction tube at A1, 201 ul
Dye tube at A3, 87 ul
Water tube at A5, 1073 ul


<div class="alert alert-warning">
    <h2>STOP!</h2>
    Before you proceed, please check:<br />
    Have you placed the required labware?<br />
    Have you put the right volume of liquid in required position?
</div>

## Instructions

In [5]:
def sample(idx):
    "Sampling the in vitro clock and seal with oil"

    # current well
    cur_well = plate.wells()[idx + SAMPLE_OFFSET]

    # pipette the dye
    transfer_viscous(pipette, protocol, calibrated_viscous(SAMPLE_VOL / 3), DYE_TUBE, cur_well)

    # pipette the sample
    aspirate_viscous(pipette, protocol, calibrated_viscous(SAMPLE_VOL), RXN_TUBE, asp_height=2)
    dispense_viscous(pipette, protocol, calibrated_viscous(SAMPLE_VOL), cur_well, if_mix=True, if_blowout=True)

def rehydrate(idx):
    "to prevent drying down"
    
    for i in range(idx):
        cur_well = plate.wells()[i + SAMPLE_OFFSET]
        
        pipette.pick_up_tip()
        pipette.aspirate(calibrated_viscous(HYDRATE_VOL), WAT_TUBE)    # just water, no oil
        dispense_viscous(pipette, protocol, calibrated_viscous(HYDRATE_VOL), cur_well, if_mix=True, if_blowout=True)

## Running

In [6]:
incubator.set_temperature(30)    # in celcius

In [7]:
# make the lists of instructions

scheduler.drop()

# sampling
time_vec = np.arange(SAMPLE_STEPS + 1) * SAMPLE_INT
func_vec = [sample] * (SAMPLE_STEPS + 1)
param_vec = [ (i,) for i in range(SAMPLE_STEPS + 1) ]
str_vec = ["Sample and mix with loading buffer"] * (SAMPLE_STEPS + 1)
n_tip_vec = [2] * (SAMPLE_STEPS + 1)
# register
scheduler.cat(time_vec, func_vec, param_vec, str_vec, n_tip_vec)

# rehydrating
# how many rehydration steps are taken per sampling step?
hydrate_steps = SAMPLE_STEPS
time_vec = (np.arange(hydrate_steps) + 1) * HYDRATE_INT
func_vec = [rehydrate] * hydrate_steps
param_vec = [ (i,) for i in np.arange(hydrate_steps) + 1 ]
str_vec = ["Rehydrating to prevent drying down"] * hydrate_steps
n_tip_vec = np.arange(hydrate_steps) + 1
# register
scheduler.cat(time_vec, func_vec, param_vec, str_vec, n_tip_vec)

# show
scheduler.report(unit="hours")

A total of 209 tips is required

At 0.00 hours, Sample and mix with loading buffer, with params (0,)
At 4.00 hours, Sample and mix with loading buffer, with params (1,)
At 8.00 hours, Sample and mix with loading buffer, with params (2,)
At 12.00 hours, Sample and mix with loading buffer, with params (3,)
At 16.00 hours, Sample and mix with loading buffer, with params (4,)
At 20.00 hours, Sample and mix with loading buffer, with params (5,)
At 24.00 hours, Sample and mix with loading buffer, with params (6,)
At 28.00 hours, Sample and mix with loading buffer, with params (7,)
At 32.00 hours, Sample and mix with loading buffer, with params (8,)
At 36.00 hours, Sample and mix with loading buffer, with params (9,)
At 40.00 hours, Sample and mix with loading buffer, with params (10,)
At 44.00 hours, Sample and mix with loading buffer, with params (11,)
At 48.00 hours, Sample and mix with loading buffer, with params (12,)
At 52.00 hours, Sample and mix with loading buffer, with params (13,)


In [ ]:
# running
log_fn = "log/20230620_ivo_no_growth.log"
scheduler.run(protocol, log_fn)